Competicion Dog Breed Identification : https://www.kaggle.com/c/dog-breed-identification

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [2]:
PATH = "data/dogbreed/"
!ls {PATH}

dogbreed.zip	sample_submission.csv	   test      train.zip
labels.csv	sample_submission.csv.zip  test.zip  Untitled.ipynb
labels.csv.zip	sub			   tmp
models		submission.csv		   train


In [3]:
sz = 224
arch = resnext101_64
bs=16
label_csv = f'{PATH}labels.csv'


In [4]:
n = len(list(open(label_csv)))-1
# random 20% indexes from range 1-n (total labels)
val_idxs = get_cv_idxs(n)

In [5]:
label_df = pd.read_csv(label_csv)
label_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [5]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, 
                       max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, 'train', 
                 f'{PATH}labels.csv', test_name='test', 
                 val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)

Podemos automatizar esto mismo en una funcion que pida sz (tamaño de la imagen) y bs (tamaño del batch)

In [6]:
def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on,
                           max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', 
               f'{PATH}labels.csv', test_name='test', num_workers=4,
               val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)
    return data if sz>300 else data.resize(340, 'tmp')

Creamos el objeto data y la CNN (la instancia learn). Establecemos precompute=True: Esto significa que durante la primera epoch (una epoch consiste en pasar por todas las imagenes del data set una vez), se calcularán las activaciones de todas las capas. ¿Por qué solo durante la primera? Empiezo por el principio .... 

Hemos cargado una arquitectura de red concreta, esto significa que ademas de tener una disposicion de capas de neuronas, tenemos los pesos de la red con unos valores concretos (aquellos que resultaron de entrenar dicha arquitectura en su momento). Esos pesos (los numeros del kernel) estaban preparados para identficar determinados patrones en las fotos donde la red se entrenó. Lo que nos interesa de la red realmente son esos pesos concretos, y de momento no los vamos a modificar.  

Cuando creamos nuestra instancia learn, estamos cargando la arquitectura arch que le pasamos como parametro menos la ultima capa, que en su lugar será construida automáticamente en funcion de las clases que necesitemos predecir (esto se sabe gracias al objeto data, que tambien se pasa como argumento). Para que esa última capa que se construye al final de la red neuronal que hemos cargado sirva de algo, tiene que tener unos datos de entrada (las activaciones de la capa anterior), por eso, debemos computar al menos una vez las activaciones de toda la red, para que haya una entrada a nuestra ultima capa. Podriamos computar las activaciones en cada nueva epoch ... pero ¿tendria esto sentido? las imagenes se repetirian en sucesivas epochs y los pesos tampoco cambiarán por tanto las activaciones (convolucion de la imagen y los pesos del kernel) produciran las mismas activaciones en la primera capa, y al convolucionar estas con los kernels de la siguiente capa (y asi sucesivamente) tampoco cambiaran las siguientes activaciones.

En resumen, hacemos precompute=True para tener una entrada a nuestra ultima capa y poder entrenar los pesos de esa ultima capa cuando hagamos learn.fit(). Computar activaciones en sucesivas epochs no cambia nada y es computacionalmente costoso

In [17]:
"""Se crea la carpeta 340 dentro de temp; dentro de 340 aparecen los subfolders models y temp, pero aun estan vacias"""
data = get_data(224, bs)
learn = ConvLearner.pretrained(arch, data, precompute=True)

Entrenamos el modelo, con learning rate = 0.01, durante 5 epochs. El por qué de ese valor de learning rate estará explicado en la libreta que habla de 'cyclical learning rates', un paper publicado en 2015, que explica como encontrar un valor apropiado para este parametro en funcion de lo rápido que decrece la función de pérdida entrenando con ese valor

In [18]:
learn.fit(0.01,5)

epoch      trn_loss   val_loss   accuracy                    
    0      0.634946   0.354258   0.897461  
    1      0.396411   0.319496   0.90332                      
    2      0.337316   0.289285   0.912598                    
    3      0.235152   0.327542   0.905599                     
    4      0.214835   0.311772   0.911133                     



[0.3117722, 0.9111328125]

In [19]:
learn.precompute = False
learn.fit(1e-2, 5, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                    
    0      0.281992   0.250768   0.919434  
    1      0.292057   0.240535   0.922363                    
    2      0.208027   0.250002   0.919922                    
    3      0.220461   0.250099   0.911133                    
    4      0.21644    0.249182   0.913574                    



[0.24918243, 0.91357421875]

overfitting (trn_loss << val_loss), osea problema de varianza o de generalizacion. Aumentaremos el tamaño de las imagenes para prevenir overfitting

Este es un buen punto para guardar. Hay que tener en cuenta lo siguiente antes: cargar un modelo da problemas si lo guardamos con precompute=True y lo cargamos con precompute=False y viceversa. Porque uno de los archivos (el que se guarda o el que se recupera) tendria activaciones distintas (precompute=False) en todas las capas de la red, y el otro tendria las activaciones (menos la de la primera capa) de la arquitectura tal cual la cargamos. Asi que guardaremos el modelo (el modelo guardado se queda en 340/tmp/models) con precompute=False

Cuando volvamos a conectar con la notebook (o hagamos restart en el kernel si se queda atascado en algun momento), es importante crear una nueva instancia del objeto learn pero con precompute=False : learn = ConvLearner.pretrained(arch, data, precompute = False)

In [20]:
learn.save('dogbreed_resnext10164_pre')

In [7]:
# encendemos de nuevo el kernel. Hacemos todos los imports y demas hasta la celda donde creamos el objeto data y learn
data = get_data(224, bs)
# preparamos una instancai de ConvLearner con precompute = False
learn = ConvLearner.pretrained(arch, data, precompute=False)
learn.load('dogbreed_resnext10164_pre')

Ya podemos seguir con el entrenamiento. Una forma de reducir overfitting es aumentando el tamaño de las imagénes: en cierto sentido son imágenes distintas que requerirán ajustar los pesos y 'despegarse' un poco de la distribucion de imagenes original para tener estas en cuenta, pero por otra parte, son imagénes muy parecidas; un perro de X raza seguirá siendo de esa raza despues de aumentar la imágen, seguirá teniendo los mismos colores, la misma forma etc, asi que esto no hará que nuestros pesos se desvien mucho de su valor actual

In [8]:
learn.set_data(get_data(299, bs))


Entrenamos durante 3 ciclos, cada uno de los cuales durará un epoch

In [9]:
learn.fit(1e-2, 3, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                    
    0      0.24987    0.240885   0.922852  
    1      0.252596   0.262771   0.915039                    
    2      0.240615   0.256994   0.920898                    



[0.25699446, 0.9208984375]

El trn_loss y val_loss se van reduciendo a la vez, esto es buena señal. Buen momento para volver a guardar el modelo, estos entrenamientos ya van tardando más ... 

In [10]:
learn.save('dogbreed_resnext10164_pre')
# a la hora de cargar, cargarlo preparando el objeto learn con todas las caracteristicas que tenia el modelo antes de hacer el save: precompute=False, data=get_data(299,bs)

In [7]:
# cargamos el modelo
data = get_data(299,bs)
learn = ConvLearner.pretrained(arch, data, precompute=False)
learn.load('dogbreed_resnext10164_pre')

Continuamos el entrenamiento: 3 ciclos, cada uno un epoch, como antes. La novedad ahora es que cada nuevo ciclo tendra 2 veces más epochs que el anterior (cycle_mult = 2). Es decir:

primer ciclo: 1 epoch
segundo ciclo : 2 * 1 = 2 epochs
tercer ciclo : 2 * 2 = 4 epochs

En total 7 epochs

In [8]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   accuracy                    
    0      0.183962   0.25741    0.925293  
    1      0.19902    0.252502   0.922363                    
    2      0.180208   0.240001   0.922852                    
    3      0.183189   0.255142   0.921387                    
    4      0.199824   0.272928   0.925781                    
    5      0.153954   0.256319   0.922363                    
    6      0.1296     0.248201   0.927572                    



[0.2482013, 0.9275716147385538]

In [9]:
learn.save('dogbreed_resnext10164_postcycle_mult')


In [7]:
data = get_data(299,bs)
learn = ConvLearner.pretrained(arch, data, precompute=False)
learn.load('dogbreed_resnext10164_postcycle_mult')

In [ ]:
"""Cuando hago TTA en un modelo, lo guardo y luego lo vuelvo a cargar, quizas para mejorarlo y cuando acabo vuelvo a hacer
TTA a la hora de conseguir predicciones tarda muchisimo en ejecutarse, asi que como ya guardé
un modelo con TTA, ahora hago las predicciones sobre el test sin ir generando versiones nuevas de fotos y predecir con una media
(que es llo que hace TTA)

Interiormente learn.predict y learn.TTA llaman a predict_with_targs (en el caso de predict, devuelve solo log_preds, en el caso
de TTA, devuelve tambien los targets)"""

Esta libreria esta construida encima de pytorch, que suele devolver el logaritmo de las predicciones en lugar de las probabilidades en sí. Por eso, una vez llamamos a predict (sobre el test set) para obetener 120 (log de) predicciones (por las 120 clases) sobre cada una de los 10357 fotos, es necesario despues hacer la exponencial de dichos logaritmos para obetener las probabilidades finales. La biblioteca numpy permite hacer esto 'element-wise' (la exponencial de cada una de las salidas de predict)

In [12]:
log_preds = learn.predict(is_test = True)

In [13]:
probs = np.exp(log_preds)


In [14]:
probs.shape

(10357, 120)

Esta biblioteca implementa tambien el metodo TTA (test time augmentation). Lo que hace esta funcion es que por cada foto del test/validation set, hace 4 transformaciones (ligeras variaciones), usa el modelo para predecir la clasificacion sobre las 5 fotos (la original y las 4 variaciones) y toma una media de la clasificacion. Es decir, basicamente vamos tomando una por una las fotos del validation/test set, y creando 4 transformaciones cada vez, teniendo las 5 en cuenta clasificamos la origianal y pasamos a la siguiente. La forma de usarla seria la misma:

log_preds, y = learn.TTA(is_test = True) <--- sobre el test set, sin este argumento seria sobre el validation set
probs = np.mean(np.exp(log_preds),0) <--- sacamos probabilidades y la media de las 5 fotos. Esto para todo el test set



Quedan cosas por explicar (encontrar un buen learning rate siguiendo el paper antes mencionado), pero abstrayendo algunos detalles y siguiendo esta implementación sin mayor complicación, se puede conseguir una posicion de 385/1286 en el momento de subirlo. 

<img src="files/score.png">